In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset 
from trackml.randomize import shuffle_hits
from trackml.score import score_event 
import os
import math
%run utils.ipynb

In [ ]:
hits, cells, particles, truth = load_data_single_event(1000)

In [ ]:
truth.head()

In [2]:
"""
    hits: tensor of samples x timesteps x features
    bounds: K:V of x_max, x_min, y_max, y_min, z_max, z_min
"""

"""
assuming input is samples x [x_val,y_val,z_val]
"""
def calc_norm(xyz, x_min, x_max, y_min, y_max, z_min, z_max):
    norm_x = ((xyz[0] - x_min)/(x_max - x_min))
    norm_y = ((xyz[1] - y_min)/(y_max - y_min))
    norm_z = ((xyz[2] - z_min)/(z_max - z_min))
    return [norm_x,norm_y,norm_z]

def norm_hits(hits, bounds):
    assert ('x_max' and 'x_min' and 'y_max' and 'y_min' and 'z_max' and 'z_min' in bounds) or ('r_max' and 'r_min' and 'phi_max' and 'phi_min' and 'z_max' and 'z_min' in bounds)
    return np.apply_along_axis(calc_norm, 1, hits, x_min = bounds['x_min'], x_max = bounds['x_max'], y_min = bounds['y_min'], y_max = bounds['y_max'], z_min = bounds['z_min'], z_max = bounds['z_max']) 


def calc_bounds():
    bounds={'x_max':0, 'x_min':0, 'y_max':0, 'y_min':0, 'z_max':0, 'z_min':0}
    for num in range(1000,1099):
        file_name = 'event00000' + str(num)
        hits, cells, particles, truth = load_event('data/train_sample/'+file_name)
        event_x_max, event_x_min, event_y_max, event_y_min, event_z_max, event_z_min = hits['x'].max(), hits['x'].min(), hits['y'].max(), hits['y'].min(), hits['z'].max(), hits['z'].min() 
        if bounds['x_max'] < event_x_max:
            bounds['x_max'] = event_x_max
        if bounds['x_min'] > event_x_min:
            bounds['x_min'] = event_x_min
        if bounds['y_max'] < event_y_max:
            bounds['y_max'] = event_y_max
        if bounds['y_min'] > event_y_min:
            bounds['y_min'] = event_y_min
        if bounds['z_max'] < event_z_max:
            bounds['z_max'] = event_z_max
        if bounds['z_min'] > event_y_min:
            bounds['z_min'] = event_y_min
    return bounds
        

"""
input: hits and truth dfs

output: df of particle_id as index and list of particles 
"""

def gen_tracks(truth_df):
    assert(isinstance(truth_df, pd.DataFrame))
    #print(truth_df)
    truth_df['dist'] = np.sqrt(truth_df['tx']**2+truth_df['ty']**2+truth_df['tz']**2)
    grouped = truth_df.groupby('particle_id')['hit_id','dist']
    a = grouped.apply(lambda x: x.sort_values(by=['dist'],ascending=True))
    final = a.groupby('particle_id')['hit_id'].apply(np.array)
    return final 

##there's still a problem with how to deal with hits that have a particle id of 0

def batch_iter(truth_df, batch_size):
    tracks = gen_tracks(truth_df).values
    np.random.shuffle(tracks) 
    remainder = len(tracks) % batch_size if len(tracks) % batch_size is not 0 else 0
    if remainder is not 0:
        modded_tracks = tracks[:-remainder]
    else:
        modded_tracks = tracks 
    assert(len(modded_tracks)%batch_size is 0)
    for batch in modded_tracks.reshape(-1,batch_size,1):
        yield batch
        
def get_data(max_seq_len, batch_size, feature_len, truth_df, hits_df):
    hits = hits_df
    max_seq_len = max_seq_len
    b_size = batch_size
    features = feature_len #xyz or phi r z
    all_data = list(batch_iter(truth_df,b_size))
    
    #print(all_data)
    for result in all_data:
        batch = []
        labels_tensor = []
        for track_list in result:
            for hit_id in track_list:
                hit_coord = []
                track = []
                for elem in hit_id:
                    x,y,z = hits.loc[hits['hit_id']== elem]['x'].item(), hits.loc[hits['hit_id']== elem]['y'].item(), hits.loc[hits['hit_id']== elem]['z'].item()
                    r,phi,z = cartesian_to_3d_polar(x,y,z)
                    hit_coord = [r,phi,z]
                    track.append(hit_coord)
                zeros_to_add = max_seq_len - len(track)
                if zeros_to_add > 0:
                    add_array = np.zeros((zeros_to_add,3))
                    np_data = np.array(track) 
                    padded_track_data  = np.append(np_data,add_array,axis=0)
                elif zeros_to_add < 0:
                    modded_track = track[:zeros_to_add]
                    padded_track_data = np.array(modded_track)
                else:
                    padded_track_data = np.array(track)
            row_label = padded_track_data[1:]
            padded_row_label = np.append(row_label, np.zeros((1,3)),axis=0)
            labels_tensor.append(padded_row_label)
            batch.append(padded_track_data)
        padded_batch_data = np.array(batch)
        padded_labels = np.array(labels_tensor)
        #print(padded_labels)
        yield padded_batch_data, padded_labels
        
def next_batch(max_seq_len, batch_size, feature_len):
    all_data = load_dataset('data/train_sample/', parts=['hits','truth'])
    for data in all_data:
        hit_df, truth_df = data[1], data[2]
        yield from get_data(max_seq_len, batch_size, feature_len, truth_df, hit_df)
        
        

In [3]:
n_steps = 7
n_input_features = 3
n_neurons = 200
n_outputs = 3
learning_rate = 0.001

batch_size = 10

X = tf.placeholder(tf.float32, [None, n_steps, n_input_features], name='input' )
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs], name='label')

#uses tanh for inner activations
lstm = tf.contrib.rnn.LSTMCell(num_units = n_neurons, use_peepholes = True)
#lstm = tf.contrib.rnn.LayerNormBasicLSTMCell(num_units = n_neurons)


lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(lstm, output_size = n_outputs)
rnn_outputs, states = tf.nn.dynamic_rnn(lstm_cell, X, dtype = tf.float32)


global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

print_op = tf.Print(rnn_outputs,[rnn_outputs], name='print_pred')
loss_diff = tf.reduce_sum(tf.square(rnn_outputs-y), 2)
mask = tf.sign(tf.reduce_max(tf.abs(y),2))
mask_loss = mask*loss_diff
batch_loss = tf.reduce_mean(mask_loss)


opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = opt.minimize(batch_loss, global_step=global_step)

init = tf.global_variables_initializer()
data_itr = next_batch(n_steps, batch_size,n_input_features)


Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
n_iterations = 500
batch_size = 50 

#saver = tf.train.Saver()
#tf.reset_default_graph()


with tf.name_scope('summaries'):
        tf.summary.scalar('loss', batch_loss)
        tf.summary.histogram('histogram_loss', batch_loss)
        summary_op = tf.summary.merge_all()
        
with tf.Session() as sess:
   
    #saver.restore(sess, "./lstm_weight1sckpt")
    init.run()
    initial_step = global_step.eval()
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./logs', sess.graph)
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = next(data_itr)
        
        
        
        
        
        _, loss_batch, summary, test_print = sess.run([train_op, batch_loss, summary_op, print_op], feed_dict={X:x_batch, y:y_batch})
        writer.add_summary(summary, global_step=iteration)
        
        if iteration %100 ==0:
            mse = batch_loss.eval(feed_dict={X:x_batch, y:y_batch})
            print(iteration, "\tMSE:", mse)
            saver.save(sess, './checkpoints', iteration)
    #saver.save(sess,"./lstm_weights1.ckpt")

INFO:tensorflow:Restoring parameters from ./checkpoints-0
0 	MSE: 11780.666
100 	MSE: 53274.473
200 	MSE: 14221.977
300 	MSE: 12399.667
400 	MSE: 6165.5723


In [ ]:
%tensorboard --logdir=./logs